# Riskiness r修改

In [1]:
import sympy
from pandas import Series,DataFrame
from scipy.optimize import fsolve
import math
import time
import pandas as pd
import numpy  as  np
import os

# 無風險利率
* 我們須扣除無風險利率，得到超額報酬率以計算Riskiness r
* 目前使用LIBOR [1個月利率](https://www.global-rates.com/interest-rates/libor/american-dollar/usd-libor-interest-rate-overnight.aspx)

In [2]:
rfrate = 2.45785 / 12 /100
rfrate

0.0020482083333333333

# 主要調動部分:

### fi()
* np.mean( np.exp(-1*arr_returns/x) ) - 1  使得其值較接近0

### get_riskiness_r ()
* 原本fsolve疊代出來的值不是我們想像的接近0，這是誤差的來源!!!!
* 現在將算出來的r回代，算出來後確認接近為0才跳脫迴圈。
* guess建議代10**(-8)

In [3]:
def f1(x,arr_returns):
    arr_returns = arr_returns
    return np.mean( np.exp(-1*arr_returns/x) ) - 1
def get_riskiness_r(guess , arr_returns):
    while (guess<10**(2)):
        risk2 = fsolve(f1,guess,arr_returns)  #引用函數f1
        print('guess:',guess,'risk:',risk2,"value:",f1(risk2,arr_returns))
        if (risk2 != guess) and (f1(risk2,arr_returns)<0.1):
            break
        guess = guess*10
    return risk2

In [4]:
data_file = './fortestriskr.csv'
df = pd.read_csv(data_file)
etf_list = list(df.columns)[1:]
df = df.iloc[:,1:]

# 計算每支ETF的riskiness r
etf_dic = {}

for etf_name in etf_list[:]:
    # 過濾平均值正負
    #if df[etf_name].mean(axis=0) < 0:
    #    continue
    arr_returns = df[etf_name] - rfrate
    guess = 10**(-8) #注意疊代
    #### 列出細項
    #print('%s指標值:  %f' %(etf_name,risk2[0]) )
    print(etf_name)
    etf_dic[etf_name] = get_riskiness_r(guess,arr_returns)
'''
#etf_name = 'CPI'
etf_name = 'MOM'
arr_returns = df[etf_name] - rfrate
#arr_returns = arr_returns*1000      #調整數量級
guess = 10**(-3)
#### 列出細項
#print('%s指標值:  %f' %(etf_name,risk2[0]) )
etf_dic[etf_name] = get_riskiness_r(guess,arr_returns)

'''
#print(etf_dic)
sorted_x = sorted(etf_dic.items(), key=lambda kv: kv[1])
    #sorted_reversed_x = list(reversed(sorted_x))

# 取前10名
top10 = sorted_x[:10]
top10

CHEP
guess: 1e-08 risk: [1.e-08] value: inf
guess: 1e-07 risk: [1.e-07] value: inf
guess: 1e-06 risk: [1.e-06] value: inf
guess: 9.999999999999999e-06 risk: [1.e-05] value: inf
guess: 9.999999999999999e-05 risk: [0.00019112] value: 5.4728749386818545e+129
guess:

C:\Users\R\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
C:\Users\R\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:163: RuntimeWarning: The number of calls to function has reached maxfev = 400.
  warnings.warn(msg, RuntimeWarning)


 0.001 risk: [1.32339162e+13] value: 2.220446049250313e-16
CPI
guess: 1e-08 risk: [1.e-08] value: inf
guess: 1e-07 risk: [1.e-07] value: inf
guess: 1e-06 risk: [1.e-06] value: inf
guess: 9.999999999999999e-06 risk: [1.e-05] value: inf
guess: 9.999999999999999e-05 risk: [4.14772066e+12] value: 0.0
FLAG
guess: 1e-08 risk: [1.e-08] value: inf
guess: 1e-07 risk: [1.e-07] value: inf
guess: 1e-06 risk: [1.e-06] value: inf
guess: 9.999999999999999e-06 risk: [1.e-05] value: inf
guess: 9.999999999999999e-05 risk: [0.00017189] value: 8.042837224772817e+164
guess: 0.001 risk: [0.10875815] value: 4.440892098500626e-16
HDG
guess: 1e-08 risk: [1.e-08] value: inf
guess: 1e-07 risk: [1.e-07] value: inf
guess: 1e-06 risk: [1.e-06] value: inf
guess: 9.999999999999999e-06 risk: [1.e-05] value: inf
guess: 9.999999999999999e-05 risk: [0.06657258] value: 0.03192654082430746
MCRO
guess: 1e-08 risk: [1.e-08] value: inf
guess: 1e-07 risk: [1.e-07] value: inf
guess: 1e-06 risk: [1.e-06] value: inf
guess: 9.9999

[('test', array([0.00745288])),
 ('MNA', array([0.05058234])),
 ('HDG', array([0.06657258])),
 ('FLAG', array([0.10875815])),
 ('QAI', array([0.15105757])),
 ('MCRO', array([0.4112832])),
 ('MRGR', array([11320612.57215443])),
 ('CPI', array([4.14772066e+12])),
 ('WTMF', array([4.39601961e+12])),
 ('RALS', array([6.42191758e+12]))]

In [5]:
etf_name = 'MCRO'
arr_returns = df[etf_name] - rfrate
f1(0.23324478,arr_returns)

0.01387906907430847

# 計算Riskiness R，並依照R值排序，傳回前十名的ETF

# 讀取全部ETF資料夾